# For creating Distance and Duration Matrix

In [1]:
!pip install folium


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import folium



In [3]:
import json
# load data using Python JSON module
with open('stops.json','r') as f:
    data = json.loads(f.read())

#All bus stops
bus_stops_df = pd.DataFrame.from_dict(data)
bus_stops_df = bus_stops_df.rename({0: 'Long', 1: 'Lat', 2: 'Name'}).T.reset_index()
bus_stops_df = bus_stops_df.rename(columns = {'index':'stop_code'})
bus_stops_df['stop_code'] = bus_stops_df['stop_code'].apply(int)


#load punggol bus stops
pung_df = pd.read_csv('punggol_bus_stops.csv')
pung_df = pung_df['stop_code']
pung_df

#merge data

pung_df = pd.merge(pung_df, bus_stops_df, on='stop_code')
pung_df

,stop_code,Long,Lat,Name
0,65009,103.90224,1.40371,Punggol Temp Int
1,65419,103.91395,1.39946,Opp Horizon Pr Sch
2,65411,103.91385,1.39945,Horizon Pr Sch
3,65429,103.91202,1.39679,Greendale Pr Sch
4,65371,103.91627,1.40289,Blk 682A
...,...,...,...,...
65,65091,103.90656,1.40398,Blk 301A
66,65099,103.90711,1.40447,Blk 604A
67,65439,103.89632,1.40173,Blk 220C
68,65431,103.89674,1.40239,Twin Waterfalls


In [4]:
#create lat_long of bus-stops in punggol
lat_long = []
lat_long_intchg = []

lat = pung_df['Lat'].to_list()
long = pung_df['Long'].to_list()
stop_code = pung_df['stop_code'].to_list()

for lat, long in zip(lat, long):
    lat_long.append([lat, long])

#lat_long of punggol interchange
#stop_code = 65009
intchg_stop_code = 65009
intchg_df = pung_df[pung_df['stop_code'] == intchg_stop_code].reset_index()
lat_long_intchg = [intchg_df.iloc[0]['Lat'], intchg_df.iloc[0]['Long']]




In [6]:
pung_df.to_csv('punggol_busstops.csv')

In [5]:
map_folium = folium.Map([1.35255,103.82580], zoom_start=11.4) # Create base map object

for i in range(0,len(lat_long)):

    lat_long_of_one_listing = lat_long[i]    # Get a pair of lat and long
    pop_display_price = str(stop_code[i])  # Get the price of listing and format the string
    tooltip = 'Click me!'

    # Generate marker and add to base map
    folium.CircleMarker(location=lat_long_of_one_listing,radius=2).add_to(map_folium)
    folium.CircleMarker(location=lat_long_intchg,radius=2, color='red').add_to(map_folium) #Interchange
display(map_folium)

In [109]:
#Create origin, destination matrix dataframe
lat_org = []
long_org = []
lat_des = []
long_des = []

for lat1,long1 in lat_long:
    for lat2, long2 in lat_long:
        lat_org.append(lat1)
        long_org.append(long1)
        lat_des.append(lat2)
        long_des.append(long2)


lat_long_df = pd.DataFrame({'origin_lat':lat_org, 'origin_long':long_org,
                            'dest_lat':lat_des, 'dest_long':long_des})
lat_long_df['origin'] = lat_long_df['origin_lat'].apply(str) + ',' + lat_long_df['origin_long'].apply(str)
lat_long_df['destination'] = lat_long_df['dest_lat'].apply(str) + ',' + lat_long_df['dest_long'].apply(str)

lat_long_df = pd.merge(lat_long_df, pung_df, left_on = ['origin_lat', 'origin_long'], 
                       right_on=['Lat', 'Long']).rename(columns={'stop_code': 'stop_code_org'})

lat_long_df = lat_long_df.drop(columns = ['Long', 'Lat', 'Name'])

lat_long_df = pd.merge(lat_long_df, pung_df, left_on = ['dest_lat', 'dest_long'], 
                       right_on=['Lat', 'Long']).rename(columns={'stop_code': 'stop_code_dest'})

lat_long_df = lat_long_df.drop(columns = ['Long', 'Lat', 'Name'])

#Create .csv dataset for distance matrix API
lat_long_dist_mtx = lat_long_df.drop(columns = ['origin_lat', 'origin_long', 'dest_lat',
                                                'dest_long', 'stop_code_org', 'stop_code_dest'])
lat_long_dist_mtx['mode'] = 'driving'
lat_long_dist_mtx['traffic_model'] = 'best_guess'
lat_long_dist_mtx['departure_time'] = 'now'

lat_long_dist_mtx.to_csv('test.csv', sep=';', index=False)


In [182]:
#After running test.csv with distancematrixAPI, use the output of the distance matrix(distance_matrix.csv)
#from API to create final distance matrix to be used for optimsation code
distance_mtx_full_df = pd.read_csv('distance_matrix_fromAPI.csv')
distance_mtx_df = distance_mtx_full_df[['origin', 'destination', 'Distance (meters)']]
distance_mtx_df = pd.merge(distance_mtx_df, lat_long_df, on=['origin', 'destination'] )
distance_mtx_df = distance_mtx_df.drop(columns=['origin_lat', 'origin_long', 'dest_lat', 'dest_long'])
stop_code_org = distance_mtx_df['stop_code_org'].to_list()
stop_code_dest = distance_mtx_df['stop_code_dest'].to_list()
distance = distance_mtx_df['Distance (meters)'].to_list()

stop_code_list1 = stop_code_org[0:70]
stop_code_list2 = stop_code_org[0:70]
distance_mtx_dict = {}
dist_list = []

for stop_code1 in stop_code_list1:
    #filter for origin
    df1 = distance_mtx_df[distance_mtx_df['stop_code_org'] == stop_code1]
    for stop_code2 in stop_code_list2:
        dist = df1[df1['stop_code_dest'] == stop_code2].iloc[0]['Distance (meters)']
        dist_list.append(dist)
        
    distance_mtx_dict[stop_code1] = dist_list
    dist_list = []
distance_mtx_final = pd.DataFrame(distance_mtx_dict, index=stop_code_list1)


In [188]:
#After running test.csv with distancematrixAPI, use the output of the duration matrix from API to 
# create final duration matrix to be used for optimisation code
duration_mtx_df = distance_mtx_full_df[['origin', 'destination', 'Duration (seconds)']]
duration_mtx_df = pd.merge(duration_mtx_df, lat_long_df, on=['origin', 'destination'] )
duration_mtx_df = duration_mtx_df.drop(columns=['origin_lat', 'origin_long', 'dest_lat', 'dest_long'])
stop_code_org = duration_mtx_df['stop_code_org'].to_list()
stop_code_dest = duration_mtx_df['stop_code_dest'].to_list()
duration = duration_mtx_df['Duration (seconds)'].to_list()

stop_code_list1 = stop_code_org[0:70]
stop_code_list2 = stop_code_org[0:70]
duration_mtx_dict = {}
dist_list = []

for stop_code1 in stop_code_list1:
    #filter for origin
    df1 = duration_mtx_df[duration_mtx_df['stop_code_org'] == stop_code1]
    for stop_code2 in stop_code_list2:
        dist = df1[df1['stop_code_dest'] == stop_code2].iloc[0]['Duration (seconds)']
        dist_list.append(dist)
        
    duration_mtx_dict[stop_code1] = dist_list
    dist_list = []
duration_mtx_final = pd.DataFrame(duration_mtx_dict, index=stop_code_list1)
duration_mtx_final = duration_mtx_final / 60
duration_mtx_final.to_csv('duration_matrix_final.csv')